# Brain Tumor Classification

In [ ]:
from google.colab import drive
drive.mount('/content/drive/',  force_remount=True)

Mounted at /content/drive/


## 1. Importing Libraries

In [ ]:
import os
import cv2
import logging
import numpy as np
from pathlib import Path
from datetime import datetime
from matplotlib import pyplot as plt


import tensorflow as tf
from tensorflow import keras

import logging
from keras.models import Sequential, Model
from keras.models import Sequential, Model
from keras.layers import Input, Dropout, Dense, GlobalAveragePooling2D, Conv2D, MaxPool2D, Flatten

from keras.applications.resnet import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.efficientnet import EfficientNetB0
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications import imagenet_utils

from keras.preprocessing.image import ImageDataGenerator

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [ ]:
!cp -r "/content/drive/MyDrive/LIFPROJET/Classification/dataset" '/content/'

Ignore Warning : A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy

In [ ]:
__file__ = '/content/drive/MyDrive/LIFPROJET/Classification/tumor_classifier.ipynb'

BASE_DIR = Path(__file__).resolve().parent

path_training = Path("/content/dataset/Training")
path_test = Path("/content/dataset/Testing")


## 2. Data Initialization

In [ ]:
X_train = [] # training images
y_train  = [] # training labels

X_test = [] # testing images
y_test = [] # testing labels

IMAGE_SIZE = 128

classes = ['glioma', 'meningioma', 'notumor', 'pituitary']


for label in classes:
    train_path = path_training / label
    list_files_training = os.listdir(str(train_path))
    
    test_path = path_test / label
    list_files_test = os.listdir(str(test_path))
    
    # Training
    for file in list_files_training:
        path_image = train_path / file
        img = cv2.imread(str(path_image),0) # 0 = gray
        
        # Produce a pseudocolored image
        img = cv2.applyColorMap(img, cv2.COLORMAP_BONE)
        
        # Resize in case of error in preprocessing
        img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
        
        X_train.append(img)
        y_train.append(classes.index(label))
    
    # Test         
    for file in list_files_test:
        path_image = test_path / file
        img = cv2.imread(str(path_image),0) # 0 = gray
        
        # Produce a pseudocolored image
        img = cv2.applyColorMap(img, cv2.COLORMAP_BONE)
        
        # Resize in case of error in preprocessing
        img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
        
        X_test.append(img)
        y_test.append(classes.index(label))

In [ ]:
# Convert to numpy array and Normalize into range 0 to 1

X_train = np.array(X_train) / 255.0
X_test = np.array(X_test) / 255.0

print(X_train.shape)
print(X_test.shape)

(5712, 128, 128, 3)
(1310, 128, 128, 3)


In [ ]:
# Shuffule images
X_train, y_train = shuffle(X_train,y_train, random_state=10) 

# Initialize tensorflow classes categorical
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

# Dividing dataset into 2 sets : training and validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=10)
print(X_val.shape)
print(y_val.shape)

(1143, 128, 128, 3)
(1143, 4)


## 3. Data Augmentation

In [ ]:
train_datagen = ImageDataGenerator(rotation_range=8,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   width_shift_range=0.05,
                                   height_shift_range=0.05,
                                   horizontal_flip=True)

train_datagen.fit(X_train)

## 4. Convutional Neural Network

### 4.1 Model Settings / Pre-trained Model

In [ ]:
mobilev2_model = MobileNetV2(weights='imagenet', # Load weights pre-trained on ImageNet.
                              include_top=False,
                              input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))

9406464/9406464 [==============================] - 0s 0us/step


### 4.2 Model Initialization

In [ ]:
classifier_model = mobilev2_model.output

classifier_model = GlobalAveragePooling2D()(classifier_model)

classifier_model = Dense(units=4, activation="softmax")(classifier_model)

classifier_model = tf.keras.Model(inputs= mobilev2_model.input, outputs= classifier_model)

### 4.3 Model Compilation

In [ ]:
PRE_TRAINED_MODEL = "MobileNetV2"
OPTIMIZER = "SGD"

LOSS = 'categorical_crossentropy'

BATCH_SIZE = 2
EPOCHS = 15

MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)


In [ ]:
classifier_model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])
#classifier_model.summary()

### 4.4 Model Training

#### 4.4.1 Save checkpoints during training

In [ ]:
checkpoint_path = "logs/checkpoints/" + "_" + MODEL_NAME
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                         save_weights_only=True,
                                                         verbose=1)


#### 4.4.2 Initialize TensorBoard during training

In [ ]:
log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 1,
                     callbacks=[tensorboard_callback])


Epoch 1/15
2285/2285 [==============================] - 58s 20ms/step - loss: 1.1408 - accuracy: 0.5080 - val_loss: 1.0042 - val_accuracy: 0.5906
Epoch 2/15
2285/2285 [==============================] - 43s 19ms/step - loss: 0.9608 - accuracy: 0.6211 - val_loss: 0.7340 - val_accuracy: 0.7297
Epoch 3/15
2285/2285 [==============================] - 44s 19ms/step - loss: 0.8987 - accuracy: 0.6612 - val_loss: 0.7379 - val_accuracy: 0.7218
Epoch 4/15
2285/2285 [==============================] - 44s 19ms/step - loss: 0.8410 - accuracy: 0.6892 - val_loss: 0.5617 - val_accuracy: 0.7909
Epoch 5/15
2285/2285 [==============================] - 44s 19ms/step - loss: 0.7851 - accuracy: 0.7196 - val_loss: 0.5513 - val_accuracy: 0.7962
Epoch 6/15
2285/2285 [==============================] - 44s 19ms/step - loss: 0.7737 - accuracy: 0.7159 - val_loss: 0.4716 - val_accuracy: 0.8320
Epoch 7/15
2285/2285 [==============================] - 44s 19ms/step - loss: 0.7392 - accuracy: 0.7380 - val_loss: 0.5140 -

## All the others training models

### MobileNetV2

In [ ]:
mobilev2_model = MobileNetV2(weights='imagenet', # Load weights pre-trained on ImageNet.
                              include_top=False,
                              input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))
classifier_model = mobilev2_model.output
classifier_model = GlobalAveragePooling2D()(classifier_model)
classifier_model = Dense(units=4, activation="softmax")(classifier_model)
classifier_model = tf.keras.Model(inputs= mobilev2_model.input, outputs= classifier_model)

PRE_TRAINED_MODEL = "MobileNetV2"
OPTIMIZER = "SGD"
LOSS = 'kl_divergence'


MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)

log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

classifier_model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])

classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 10,
                     callbacks=[tensorboard_callback])


Epoch 1/15
Epoch 2/15
Epoch 3/15
Epoch 4/15
Epoch 5/15
Epoch 6/15
Epoch 7/15
Epoch 8/15
Epoch 9/15
Epoch 10/15
Epoch 11/15
Epoch 12/15
Epoch 13/15
Epoch 14/15
Epoch 15/15


In [ ]:
mobilev2_model = MobileNetV2(weights='imagenet', # Load weights pre-trained on ImageNet.
                              include_top=False,
                              input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))

classifier_model = mobilev2_model.output
classifier_model = GlobalAveragePooling2D()(classifier_model)
classifier_model = Dense(units=4, activation="softmax")(classifier_model)
classifier_model = tf.keras.Model(inputs= mobilev2_model.input, outputs= classifier_model)

PRE_TRAINED_MODEL = "MobileNetV2"
OPTIMIZER = "SGD"
LOSS = 'categorical_hinge'


MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)

log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

classifier_model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])

classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 10,
                     callbacks=[tensorboard_callback])


Epoch 1/15
Epoch 2/15
Epoch 3/15
Epoch 4/15
Epoch 5/15
Epoch 6/15
Epoch 7/15
Epoch 8/15
Epoch 9/15
Epoch 10/15
Epoch 11/15
Epoch 12/15
Epoch 13/15
Epoch 14/15
Epoch 15/15


In [ ]:
mobilev2_model = MobileNetV2(weights='imagenet', # Load weights pre-trained on ImageNet.
                              include_top=False,
                              input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))
classifier_model = mobilev2_model.output
classifier_model = GlobalAveragePooling2D()(classifier_model)
classifier_model = Dense(units=4, activation="softmax")(classifier_model)
classifier_model = tf.keras.Model(inputs= mobilev2_model.input, outputs= classifier_model)

PRE_TRAINED_MODEL = "MobileNetV2"
OPTIMIZER = "Adam"
LOSS = 'categorical_crossentropy'


MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)

log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

classifier_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])

classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 0,
                     callbacks=[tensorboard_callback])


mobilev2_model = MobileNetV2(weights='imagenet', # Load weights pre-trained on ImageNet.
                              include_top=False,
                              input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))
classifier_model = mobilev2_model.output
classifier_model = GlobalAveragePooling2D()(classifier_model)
classifier_model = Dense(units=4, activation="softmax")(classifier_model)
classifier_model = tf.keras.Model(inputs= mobilev2_model.input, outputs= classifier_model)

PRE_TRAINED_MODEL = "MobileNetV2"
OPTIMIZER = "Adam"
LOSS = "kl_divergence"
MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)

log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

classifier_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])

classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 0,
                     callbacks=[tensorboard_callback])





mobilev2_model = MobileNetV2(weights='imagenet', # Load weights pre-trained on ImageNet.
                              include_top=False,
                              input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))
classifier_model = mobilev2_model.output
classifier_model = GlobalAveragePooling2D()(classifier_model)
classifier_model = Dense(units=4, activation="softmax")(classifier_model)
classifier_model = tf.keras.Model(inputs= mobilev2_model.input, outputs= classifier_model)

PRE_TRAINED_MODEL = "MobileNetV2"
OPTIMIZER = "Adam"
LOSS = "categorical_hinge"
MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)

log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

classifier_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])

classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 0,
                     callbacks=[tensorboard_callback])

In [ ]:
mobilev2_model = MobileNetV2(weights='imagenet', # Load weights pre-trained on ImageNet.
                              include_top=False,
                              input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))
classifier_model = mobilev2_model.output
classifier_model = GlobalAveragePooling2D()(classifier_model)
classifier_model = Dense(units=4, activation="softmax")(classifier_model)
classifier_model = tf.keras.Model(inputs= mobilev2_model.input, outputs= classifier_model)

PRE_TRAINED_MODEL = "MobileNetV2"
OPTIMIZER = "NAdam"
LOSS = 'categorical_crossentropy'



MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)

log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

classifier_model.compile(optimizer=keras.optimizers.Nadam(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])

classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 0,
                     callbacks=[tensorboard_callback])

In [ ]:
mobilev2_model = MobileNetV2(weights='imagenet', # Load weights pre-trained on ImageNet.
                              include_top=False,
                              input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))
classifier_model = mobilev2_model.output
classifier_model = GlobalAveragePooling2D()(classifier_model)
classifier_model = Dense(units=4, activation="softmax")(classifier_model)
classifier_model = tf.keras.Model(inputs= mobilev2_model.input, outputs= classifier_model)

PRE_TRAINED_MODEL = "MobileNetV2"
OPTIMIZER = "NAdam"
LOSS = "kl_divergence"
MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)

log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

classifier_model.compile(optimizer=keras.optimizers.Nadam(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])

classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 0,
                     callbacks=[tensorboard_callback])





mobilev2_model = MobileNetV2(weights='imagenet', # Load weights pre-trained on ImageNet.
                              include_top=False,
                              input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))
classifier_model = mobilev2_model.output
classifier_model = GlobalAveragePooling2D()(classifier_model)
classifier_model = Dense(units=4, activation="softmax")(classifier_model)
classifier_model = tf.keras.Model(inputs= mobilev2_model.input, outputs= classifier_model)

PRE_TRAINED_MODEL = "MobileNetV2"
OPTIMIZER = "NAdam"
LOSS = "categorical_hinge"
MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)

log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

classifier_model.compile(optimizer=keras.optimizers.Nadam(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])

classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 0,
                     callbacks=[tensorboard_callback])

In [ ]:

#### ResNet

In [ ]:
resnet_model = ResNet50(weights='imagenet', # Load weights pre-trained on ImageNet.
                       include_top=False,
                       input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))

classifier_model = resnet_model.output
classifier_model = GlobalAveragePooling2D()(classifier_model)
classifier_model = Dense(units=4, activation="softmax")(classifier_model)
classifier_model = tf.keras.Model(inputs= resnet_model.input, outputs= classifier_model)

PRE_TRAINED_MODEL = "ResNet"
OPTIMIZER = "Adam"
LOSS = "categorical_crossentropy"
MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)

log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

classifier_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])

classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 0,
                     callbacks=[tensorboard_callback])

#---------------------------------------------------------------
resnet_model = ResNet50(weights='imagenet', # Load weights pre-trained on ImageNet.
                       include_top=False,
                       input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))
classifier_model = resnet_model.output
classifier_model = GlobalAveragePooling2D()(classifier_model)
classifier_model = Dense(units=4, activation="softmax")(classifier_model)
classifier_model = tf.keras.Model(inputs= resnet_model.input, outputs= classifier_model)

PRE_TRAINED_MODEL = "ResNet"
OPTIMIZER = "Adam"
LOSS = "kl_divergence"
MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)

log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

classifier_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])

classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 0,
                     callbacks=[tensorboard_callback])
#--------------------------------------------------------------
resnet_model = ResNet50(weights='imagenet', # Load weights pre-trained on ImageNet.
                       include_top=False,
                       input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))
classifier_model = resnet_model.output
classifier_model = GlobalAveragePooling2D()(classifier_model)
classifier_model = Dense(units=4, activation="softmax")(classifier_model)
classifier_model = tf.keras.Model(inputs= resnet_model.input, outputs= classifier_model)

PRE_TRAINED_MODEL = "ResNet"
OPTIMIZER = "Adam"
LOSS = "categorical_hinge"
MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)

log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

classifier_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])

classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 0,
                     callbacks=[tensorboard_callback])




#--------------------------------------------------------------
#--------------------------------------------------------------
#--------------------------------------------------------------
resnet_model = ResNet50(weights='imagenet', # Load weights pre-trained on ImageNet.
                       include_top=False,
                       input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))
classifier_model = resnet_model.output
classifier_model = GlobalAveragePooling2D()(classifier_model)
classifier_model = Dense(units=4, activation="softmax")(classifier_model)
classifier_model = tf.keras.Model(inputs= resnet_model.input, outputs= classifier_model)

PRE_TRAINED_MODEL = "ResNet"
OPTIMIZER = "NAdam"
LOSS = "categorical_crossentropy"
MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)

log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

classifier_model.compile(optimizer=keras.optimizers.Nadam(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])

classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 0,
                     callbacks=[tensorboard_callback])

#---------------------------------------------------------------
resnet_model = ResNet50(weights='imagenet', # Load weights pre-trained on ImageNet.
                       include_top=False,
                       input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))
classifier_model = resnet_model.output
classifier_model = GlobalAveragePooling2D()(classifier_model)
classifier_model = Dense(units=4, activation="softmax")(classifier_model)
classifier_model = tf.keras.Model(inputs= resnet_model.input, outputs= classifier_model)

PRE_TRAINED_MODEL = "ResNet"
OPTIMIZER = "NAdam"
LOSS = "kl_divergence"
MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)

log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

classifier_model.compile(optimizer=keras.optimizers.Nadam(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])

classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 0,
                     callbacks=[tensorboard_callback])
#--------------------------------------------------------------
resnet_model = ResNet50(weights='imagenet', # Load weights pre-trained on ImageNet.
                       include_top=False,
                       input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))
classifier_model = resnet_model.output
classifier_model = GlobalAveragePooling2D()(classifier_model)
classifier_model = Dense(units=4, activation="softmax")(classifier_model)
classifier_model = tf.keras.Model(inputs= resnet_model.input, outputs= classifier_model)

PRE_TRAINED_MODEL = "ResNet"
OPTIMIZER = "NAdam"
LOSS = "categorical_hinge"
MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)

log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

classifier_model.compile(optimizer=keras.optimizers.Nadam(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])

classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 0,
                     callbacks=[tensorboard_callback])


#--------------------------------------------------------------
#--------------------------------------------------------------
#--------------------------------------------------------------
resnet_model = ResNet50(weights='imagenet', # Load weights pre-trained on ImageNet.
                       include_top=False,
                       input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))
classifier_model = resnet_model.output
classifier_model = GlobalAveragePooling2D()(classifier_model)
classifier_model = Dense(units=4, activation="softmax")(classifier_model)
classifier_model = tf.keras.Model(inputs= resnet_model.input, outputs= classifier_model)

PRE_TRAINED_MODEL = "ResNet"
OPTIMIZER = "SGD"
LOSS = "categorical_crossentropy"
MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)

log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

classifier_model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])

classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 0,
                     callbacks=[tensorboard_callback])

#---------------------------------------------------------------
resnet_model = ResNet50(weights='imagenet', # Load weights pre-trained on ImageNet.
                       include_top=False,
                       input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))
classifier_model = resnet_model.output
classifier_model = GlobalAveragePooling2D()(classifier_model)
classifier_model = Dense(units=4, activation="softmax")(classifier_model)
classifier_model = tf.keras.Model(inputs= resnet_model.input, outputs= classifier_model)

PRE_TRAINED_MODEL = "ResNet"
OPTIMIZER = "SGD"
LOSS = "kl_divergence"
MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)

log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

classifier_model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])

classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 0,
                     callbacks=[tensorboard_callback])
#--------------------------------------------------------------
resnet_model = ResNet50(weights='imagenet', # Load weights pre-trained on ImageNet.
                       include_top=False,
                       input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))
classifier_model = resnet_model.output
classifier_model = GlobalAveragePooling2D()(classifier_model)
classifier_model = Dense(units=4, activation="softmax")(classifier_model)
classifier_model = tf.keras.Model(inputs= resnet_model.input, outputs= classifier_model)

PRE_TRAINED_MODEL = "ResNet"
OPTIMIZER = "SGD"
LOSS = "categorical_hinge"
MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)

log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

classifier_model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])

classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 0,
                     callbacks=[tensorboard_callback])

94765736/94765736 [==============================] - 5s 0us/step


### VGG16

In [ ]:
vgg_model = VGG16(weights='imagenet', # Load weights pre-trained on ImageNet.
                       include_top=False,
                       input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))

classifier_model = vgg_model.output
classifier_model = GlobalAveragePooling2D()(classifier_model)
classifier_model = Dense(units=4, activation="softmax")(classifier_model)
classifier_model = tf.keras.Model(inputs= vgg_model.input, outputs= classifier_model)

PRE_TRAINED_MODEL = "VGG16"
OPTIMIZER = "Adam"
LOSS = "categorical_crossentropy"
MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)

log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

classifier_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])

classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 1,
                     callbacks=[tensorboard_callback])

#---------------------------------------------------------------
vgg_model = VGG16(weights='imagenet', # Load weights pre-trained on ImageNet.
                       include_top=False,
                       input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))
classifier_model = vgg_model.output
classifier_model = GlobalAveragePooling2D()(classifier_model)
classifier_model = Dense(units=4, activation="softmax")(classifier_model)
classifier_model = tf.keras.Model(inputs= vgg_model.input, outputs= classifier_model)

PRE_TRAINED_MODEL = "VGG16"
OPTIMIZER = "Adam"
LOSS = "kl_divergence"
MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)

log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

classifier_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])

classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 0,
                     callbacks=[tensorboard_callback])
#--------------------------------------------------------------
vgg_model = VGG16(weights='imagenet', # Load weights pre-trained on ImageNet.
                       include_top=False,
                       input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))
classifier_model = vgg_model.output
classifier_model = GlobalAveragePooling2D()(classifier_model)
classifier_model = Dense(units=4, activation="softmax")(classifier_model)
classifier_model = tf.keras.Model(inputs= vgg_model.input, outputs= classifier_model)

PRE_TRAINED_MODEL = "VGG16"
OPTIMIZER = "Adam"
LOSS = "categorical_hinge"
MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)

log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

classifier_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])

classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 0,
                     callbacks=[tensorboard_callback])




#--------------------------------------------------------------
#--------------------------------------------------------------
#--------------------------------------------------------------
vgg_model = VGG16(weights='imagenet', # Load weights pre-trained on ImageNet.
                       include_top=False,
                       input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))
classifier_model = vgg_model.output
classifier_model = GlobalAveragePooling2D()(classifier_model)
classifier_model = Dense(units=4, activation="softmax")(classifier_model)
classifier_model = tf.keras.Model(inputs= vgg_model.input, outputs= classifier_model)

PRE_TRAINED_MODEL = "VGG16"
OPTIMIZER = "NAdam"
LOSS = "categorical_crossentropy"
MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)

log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

classifier_model.compile(optimizer=keras.optimizers.Nadam(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])

classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 0,
                     callbacks=[tensorboard_callback])

#---------------------------------------------------------------
vgg_model = VGG16(weights='imagenet', # Load weights pre-trained on ImageNet.
                       include_top=False,
                       input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))
classifier_model = vgg_model.output
classifier_model = GlobalAveragePooling2D()(classifier_model)
classifier_model = Dense(units=4, activation="softmax")(classifier_model)
classifier_model = tf.keras.Model(inputs= vgg_model.input, outputs= classifier_model)

PRE_TRAINED_MODEL = "VGG16"
OPTIMIZER = "NAdam"
LOSS = "kl_divergence"
MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)

log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

classifier_model.compile(optimizer=keras.optimizers.Nadam(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])

classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 0,
                     callbacks=[tensorboard_callback])
#--------------------------------------------------------------
vgg_model = VGG16(weights='imagenet', # Load weights pre-trained on ImageNet.
                       include_top=False,
                       input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))
classifier_model = vgg_model.output
classifier_model = GlobalAveragePooling2D()(classifier_model)
classifier_model = Dense(units=4, activation="softmax")(classifier_model)
classifier_model = tf.keras.Model(inputs= vgg_model.input, outputs= classifier_model)

PRE_TRAINED_MODEL = "VGG16"
OPTIMIZER = "NAdam"
LOSS = "categorical_hinge"
MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)

log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

classifier_model.compile(optimizer=keras.optimizers.Nadam(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])

classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 0,
                     callbacks=[tensorboard_callback])


#--------------------------------------------------------------
#--------------------------------------------------------------
#--------------------------------------------------------------
vgg_model = VGG16(weights='imagenet', # Load weights pre-trained on ImageNet.
                       include_top=False,
                       input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))
classifier_model = vgg_model.output
classifier_model = GlobalAveragePooling2D()(classifier_model)
classifier_model = Dense(units=4, activation="softmax")(classifier_model)
classifier_model = tf.keras.Model(inputs= vgg_model.input, outputs= classifier_model)

PRE_TRAINED_MODEL = "VGG16"
OPTIMIZER = "SGD"
LOSS = "categorical_crossentropy"
MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)

log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

classifier_model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])

classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 0,
                     callbacks=[tensorboard_callback])

#---------------------------------------------------------------
vgg_model = VGG16(weights='imagenet', # Load weights pre-trained on ImageNet.
                       include_top=False,
                       input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))
classifier_model = vgg_model.output
classifier_model = GlobalAveragePooling2D()(classifier_model)
classifier_model = Dense(units=4, activation="softmax")(classifier_model)
classifier_model = tf.keras.Model(inputs= vgg_model.input, outputs= classifier_model)

PRE_TRAINED_MODEL = "VGG16"
OPTIMIZER = "SGD"
LOSS = "kl_divergence"
MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)

log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

classifier_model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])

classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 0,
                     callbacks=[tensorboard_callback])
#--------------------------------------------------------------
vgg_model = VGG16(weights='imagenet', # Load weights pre-trained on ImageNet.
                       include_top=False,
                       input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))
classifier_model = vgg_model.output
classifier_model = GlobalAveragePooling2D()(classifier_model)
classifier_model = Dense(units=4, activation="softmax")(classifier_model)
classifier_model = tf.keras.Model(inputs= vgg_model.input, outputs= classifier_model)

PRE_TRAINED_MODEL = "VGG16"
OPTIMIZER = "SGD"
LOSS = "categorical_hinge"
MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)

log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

classifier_model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])

classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 0,
                     callbacks=[tensorboard_callback])

58889256/58889256 [==============================] - 3s 0us/step
Epoch 1/15
2285/2285 [==============================] - 23s 9ms/step - loss: 0.9359 - accuracy: 0.5877 - val_loss: 0.5686 - val_accuracy: 0.7717
Epoch 2/15
2285/2285 [==============================] - 20s 9ms/step - loss: 0.5171 - accuracy: 0.8113 - val_loss: 0.3222 - val_accuracy: 0.8871
Epoch 3/15
2285/2285 [==============================] - 20s 9ms/step - loss: 0.3611 - accuracy: 0.8774 - val_loss: 0.2293 - val_accuracy: 0.9160
Epoch 4/15
2285/2285 [==============================] - 20s 9ms/step - loss: 0.2729 - accuracy: 0.9079 - val_loss: 0.2610 - val_accuracy: 0.9090
Epoch 5/15
2285/2285 [==============================] - 20s 9ms/step - loss: 0.2328 - accuracy: 0.9212 - val_loss: 0.3291 - val_accuracy: 0.8898
Epoch 6/15
2285/2285 [==============================] - 21s 9ms/step - loss: 0.2042 - accuracy: 0.9339 - val_loss: 0.2695 - val_accuracy: 0.9283
Epoch 7/15
2285/2285 [==============================] - 21s 9ms/s

### EfficientNetB0

In [ ]:
eff_model = EfficientNetB0(weights='imagenet', # Load weights pre-trained on ImageNet.
                      include_top=False,
                      input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))

classifier_model = eff_model.output
classifier_model = GlobalAveragePooling2D()(classifier_model)
classifier_model = Dense(units=4, activation="softmax")(classifier_model)
classifier_model = tf.keras.Model(inputs= eff_model.input, outputs= classifier_model)

PRE_TRAINED_MODEL = "EfficientNetB0"
OPTIMIZER = "Adam"
LOSS = "categorical_crossentropy"
MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)

log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

classifier_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])

classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 1,
                     callbacks=[tensorboard_callback])

#---------------------------------------------------------------
eff_model = EfficientNetB0(weights='imagenet', # Load weights pre-trained on ImageNet.
                      include_top=False,
                      input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))
classifier_model = eff_model.output
classifier_model = GlobalAveragePooling2D()(classifier_model)
classifier_model = Dense(units=4, activation="softmax")(classifier_model)
classifier_model = tf.keras.Model(inputs= eff_model.input, outputs= classifier_model)

PRE_TRAINED_MODEL = "EfficientNetB0"
OPTIMIZER = "Adam"
LOSS = "kl_divergence"
MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)

log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

classifier_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])

classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 0,
                     callbacks=[tensorboard_callback])
#--------------------------------------------------------------
eff_model = EfficientNetB0(weights='imagenet', # Load weights pre-trained on ImageNet.
                      include_top=False,
                      input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))
classifier_model = eff_model.output
classifier_model = GlobalAveragePooling2D()(classifier_model)
classifier_model = Dense(units=4, activation="softmax")(classifier_model)
classifier_model = tf.keras.Model(inputs= eff_model.input, outputs= classifier_model)

PRE_TRAINED_MODEL = "EfficientNetB0"
OPTIMIZER = "Adam"
LOSS = "categorical_hinge"
MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)

log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

classifier_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])

classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 0,
                     callbacks=[tensorboard_callback])




#--------------------------------------------------------------
#--------------------------------------------------------------
#--------------------------------------------------------------
eff_model = EfficientNetB0(weights='imagenet', # Load weights pre-trained on ImageNet.
                      include_top=False,
                      input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))
classifier_model = eff_model.output
classifier_model = GlobalAveragePooling2D()(classifier_model)
classifier_model = Dense(units=4, activation="softmax")(classifier_model)
classifier_model = tf.keras.Model(inputs= eff_model.input, outputs= classifier_model)

PRE_TRAINED_MODEL = "EfficientNetB0"
OPTIMIZER = "NAdam"
LOSS = "categorical_crossentropy"
MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)

log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

classifier_model.compile(optimizer=keras.optimizers.Nadam(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])

classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 0,
                     callbacks=[tensorboard_callback])

#---------------------------------------------------------------
eff_model = EfficientNetB0(weights='imagenet', # Load weights pre-trained on ImageNet.
                      include_top=False,
                      input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))
classifier_model = eff_model.output
classifier_model = GlobalAveragePooling2D()(classifier_model)
classifier_model = Dense(units=4, activation="softmax")(classifier_model)
classifier_model = tf.keras.Model(inputs= eff_model.input, outputs= classifier_model)

PRE_TRAINED_MODEL = "EfficientNetB0"
OPTIMIZER = "NAdam"
LOSS = "kl_divergence"
MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)

log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

classifier_model.compile(optimizer=keras.optimizers.Nadam(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])

classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 0,
                     callbacks=[tensorboard_callback])
#--------------------------------------------------------------
eff_model = EfficientNetB0(weights='imagenet', # Load weights pre-trained on ImageNet.
                      include_top=False,
                      input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))
classifier_model = eff_model.output
classifier_model = GlobalAveragePooling2D()(classifier_model)
classifier_model = Dense(units=4, activation="softmax")(classifier_model)
classifier_model = tf.keras.Model(inputs= eff_model.input, outputs= classifier_model)

PRE_TRAINED_MODEL = "EfficientNetB0"
OPTIMIZER = "NAdam"
LOSS = "categorical_hinge"
MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)

log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

classifier_model.compile(optimizer=keras.optimizers.Nadam(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])

classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 0,
                     callbacks=[tensorboard_callback])


#--------------------------------------------------------------
#--------------------------------------------------------------
#--------------------------------------------------------------
eff_model = EfficientNetB0(weights='imagenet', # Load weights pre-trained on ImageNet.
                      include_top=False,
                      input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))
classifier_model = eff_model.output
classifier_model = GlobalAveragePooling2D()(classifier_model)
classifier_model = Dense(units=4, activation="softmax")(classifier_model)
classifier_model = tf.keras.Model(inputs= eff_model.input, outputs= classifier_model)

PRE_TRAINED_MODEL = "EfficientNetB0"
OPTIMIZER = "SGD"
LOSS = "categorical_crossentropy"
MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)

log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

classifier_model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])

classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 0,
                     callbacks=[tensorboard_callback])

#---------------------------------------------------------------
eff_model = EfficientNetB0(weights='imagenet', # Load weights pre-trained on ImageNet.
                      include_top=False,
                      input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))
classifier_model = eff_model.output
classifier_model = GlobalAveragePooling2D()(classifier_model)
classifier_model = Dense(units=4, activation="softmax")(classifier_model)
classifier_model = tf.keras.Model(inputs= eff_model.input, outputs= classifier_model)

PRE_TRAINED_MODEL = "EfficientNetB0"
OPTIMIZER = "SGD"
LOSS = "kl_divergence"
MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)

log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

classifier_model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])

classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 0,
                     callbacks=[tensorboard_callback])
#--------------------------------------------------------------
eff_model = EfficientNetB0(weights='imagenet', # Load weights pre-trained on ImageNet.
                      include_top=False,
                      input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))
classifier_model = eff_model.output
classifier_model = GlobalAveragePooling2D()(classifier_model)
classifier_model = Dense(units=4, activation="softmax")(classifier_model)
classifier_model = tf.keras.Model(inputs= eff_model.input, outputs= classifier_model)

PRE_TRAINED_MODEL = "EfficientNetB0"
OPTIMIZER = "SGD"
LOSS = "categorical_hinge"
MODEL_NAME = PRE_TRAINED_MODEL + "_" + OPTIMIZER + "_" + LOSS + "_" + str(EPOCHS)

log_dir = "/content/drive/MyDrive/LIFPROJET/Classification/logs/fit/" + "_" + MODEL_NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

classifier_model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.0001),
              loss = LOSS,
              metrics=['accuracy'])

classifier_model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                     validation_data = (X_val,y_val),
                     epochs = EPOCHS,
                     verbose = 0,
                     callbacks=[tensorboard_callback])

16705208/16705208 [==============================] - 2s 0us/step
Epoch 1/15
2285/2285 [==============================] - 104s 41ms/step - loss: 0.6896 - accuracy: 0.7417 - val_loss: 15.9212 - val_accuracy: 0.2983
Epoch 2/15
2285/2285 [==============================] - 92s 40ms/step - loss: 0.3174 - accuracy: 0.8884 - val_loss: 1.7505 - val_accuracy: 0.2633
Epoch 3/15
2285/2285 [==============================] - 93s 41ms/step - loss: 0.2189 - accuracy: 0.9179 - val_loss: 1.2580 - val_accuracy: 0.3823
Epoch 4/15
2285/2285 [==============================] - 93s 41ms/step - loss: 0.1752 - accuracy: 0.9398 - val_loss: 1.3350 - val_accuracy: 0.3010
Epoch 5/15
2285/2285 [==============================] - 92s 40ms/step - loss: 0.1488 - accuracy: 0.9464 - val_loss: 7.1838 - val_accuracy: 0.3097
Epoch 6/15
2285/2285 [==============================] - 93s 41ms/step - loss: 0.1310 - accuracy: 0.9545 - val_loss: 1.3528 - val_accuracy: 0.4462
Epoch 7/15
2285/2285 [==============================] - 9